In [ ]:
!pip install xgboost

In [ ]:
!pip install feature-engine

In [ ]:
!pip install lazypredict

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/forest-cover-type-prediction/train.csv')
data.head(5)

# Custom Transformers

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class DistanceTransformer(BaseEstimator, TransformerMixin):
    # TODO create a tranformer that do its for any numeric variables in a pandas dataframe
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['EuclideanDistanceHidroloy'] = np.around(
            np.sqrt(X['Horizontal_Distance_To_Hydrology'] **2 +
                    X['Vertical_Distance_To_Hydrology'] **2), 
            4)
        X['Elevation_Vertical_Hydro_Minus'] = X['Elevation'] - X['Vertical_Distance_To_Hydrology']
        X['Elevation_Vertical_Hydro_Plus'] = X['Elevation'] + X['Vertical_Distance_To_Hydrology']
        X['Elevation_Vertical_Hydro_Prod'] = X['Elevation'] * X['Vertical_Distance_To_Hydrology']
        
        X['Elevation_Horizontal_Hydro_Minus'] = X['Elevation'] - X['Horizontal_Distance_To_Hydrology']
        X['Elevation_Horizontal_Hydro_Plus'] = X['Elevation'] + X['Horizontal_Distance_To_Hydrology']
        X['Elevation_Horizontal_Hydro_Prod'] = X['Elevation'] * X['Horizontal_Distance_To_Hydrology']
        
        X['Elevation_Horizontal_Fire_Minus'] = X['Elevation'] - X['Horizontal_Distance_To_Fire_Points']
        X['Elevation_Horizontal_Fire_Plus'] = X['Elevation'] + X['Horizontal_Distance_To_Fire_Points']
        X['Elevation_Horizontal_Fire_Prod'] = X['Elevation'] * X['Horizontal_Distance_To_Fire_Points']
        
        X['Elevation_Horizontal_Roadways_Minus'] = X['Elevation'] - X['Horizontal_Distance_To_Roadways']
        X['Elevation_Horizontal_Roadways_Plus'] = X['Elevation'] + X['Horizontal_Distance_To_Roadways']
        X['Elevation_Horizontal_Roadways_Prod'] = X['Elevation'] * X['Horizontal_Distance_To_Roadways']
                
        X['Hidrology_Horizonal_Fire_Minus'] = X['Horizontal_Distance_To_Hydrology'] - X['Horizontal_Distance_To_Fire_Points']
        X['Hidrology_Horizonal_Fire_Plus'] = X['Horizontal_Distance_To_Hydrology'] + X['Horizontal_Distance_To_Fire_Points']
        X['Hidrology_Horizonal_Fire_Prod'] = X['Horizontal_Distance_To_Hydrology'] * X['Horizontal_Distance_To_Fire_Points']
        
        X['Hidrology_Horizonal_Roadways_Minus'] = X['Horizontal_Distance_To_Hydrology'] - X['Horizontal_Distance_To_Roadways']
        X['Hidrology_Horizonal_Roadways_Plus'] = X['Horizontal_Distance_To_Hydrology'] + X['Horizontal_Distance_To_Roadways']
        X['Hidrology_Horizonal_Roadways_Prod'] = X['Horizontal_Distance_To_Hydrology'] * X['Horizontal_Distance_To_Roadways']
        
        X['Hidrology_Horizonal_Vertical_Minus'] = X['Horizontal_Distance_To_Hydrology'] - X['Vertical_Distance_To_Hydrology']
        X['Hidrology_Horizonal_Vertical_Plus'] = X['Horizontal_Distance_To_Hydrology'] + X['Vertical_Distance_To_Hydrology']
        X['Hidrology_Horizonal_Vertical_Prod'] = X['Horizontal_Distance_To_Hydrology'] * X['Vertical_Distance_To_Hydrology']
        
        X['Hidrology_Vertical_Fire_Minus'] = X['Vertical_Distance_To_Hydrology'] - X['Horizontal_Distance_To_Fire_Points']
        X['Hidrology_Vertical_Fire_Plus'] = X['Vertical_Distance_To_Hydrology'] + X['Horizontal_Distance_To_Fire_Points']
        X['Hidrology_Vertical_Fire_Prod'] = X['Vertical_Distance_To_Hydrology'] * X['Horizontal_Distance_To_Fire_Points']
        
        X['Hidrology_Vertical_Fire_Minus'] = X['Vertical_Distance_To_Hydrology'] - X['Horizontal_Distance_To_Roadways']
        X['Hidrology_Vertical_Fire_Plus'] = X['Vertical_Distance_To_Hydrology'] + X['Horizontal_Distance_To_Roadways']
        X['Hidrology_Vertical_Fire_Prod'] = X['Vertical_Distance_To_Hydrology'] * X['Horizontal_Distance_To_Roadways']
        
        X.drop(['Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Horizontal_Distance_To_Fire_Points'], axis=1, inplace=True)
        return X

In [ ]:
class DropIdentifierFeatures(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X.drop('Id', axis=1, inplace=True)
        return X

In [ ]:
from sklearn.utils.validation import check_is_fitted

class FromDummiesToCategories(BaseEstimator, TransformerMixin):
    def __init__(self, cols_to_operate, new_column_name):
        self.cols_to_operate = cols_to_operate
        self.new_column_name = new_column_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X1 = pd.DataFrame(X[self.cols_to_operate])
        serie = X1.columns[np.where(X1!=0)[1]]
        X[self.new_column_name] = serie
        X.drop(self.cols_to_operate, axis=1, inplace=True)
        return X

In [ ]:
class AspectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, column, new_name):
        self.column = column
        self.new_name = new_name
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X[self.new_name] = X[self.column].copy().apply(self._reclassify_aspect)
        X.drop(self.column, axis=1, inplace=True)
        return X
        
    def _reclassify_aspect(self, x):
        if x<0:
            return 'Flat'
        if  0 >= x < 45:
            return 'North'
        if 45 >= x < 90:
            return 'North_East'
        if 90 >= x < 135:
            return 'East'
        if 135 >= x < 180:
            return 'South_East'
        if 180 >= x < 225:
            return 'South'
        if 225 >= x < 270:
            return 'South_West'
        if 270 >= x < 315:
            return 'West'
        if 315 >= x <360:
            return 'North_West'
        if 360 >= x :
            return 'North_West'

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
X = data.drop('Cover_Type', axis=1).copy()
y = data['Cover_Type'].copy()
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
soil_columns = [x for x in X.columns if x.startswith('Soil_Type')]
wilder_columns = [x for x in X.columns if x.startswith('Wilder')]

# Pipeline 1

In [ ]:
from feature_engine.discretisation import EqualWidthDiscretiser, EqualFrequencyDiscretiser
from feature_engine.selection  import DropConstantFeatures, DropDuplicateFeatures, DropCorrelatedFeatures, SelectBySingleFeaturePerformance, RecursiveFeatureElimination, SmartCorrelatedSelection, DropFeatures 
from feature_engine.encoding import RareLabelEncoder, OneHotEncoder, OrdinalEncoder
from feature_engine.outliers import Winsorizer
from feature_engine.creation import MathematicalCombination
from sklearn.pipeline import Pipeline

In [ ]:
pipeline_list_1 = [
    ('dropuniquefeatures', DropIdentifierFeatures()),
    ('aspect', AspectTransformer(column='Aspect', new_name='Orientation')),
    ('soil_columns_dummies', FromDummiesToCategories(cols_to_operate=soil_columns, new_column_name='Soil_Type')),
    ('wilder_columns_dummies', FromDummiesToCategories(cols_to_operate=wilder_columns, new_column_name='Wilderness')),
    ('soil_rare', RareLabelEncoder(tol=0.05, variables=['Soil_Type'])),
    ('dp', DropConstantFeatures(tol=0.99)),
    ('dd', DropDuplicateFeatures()),
    ('dt', DistanceTransformer()),
     ('one_hot',OneHotEncoder(variables=['Soil_Type', 'Wilderness', 'Orientation'])),
    ('dteq', EqualFrequencyDiscretiser(q=10, variables=['EuclideanDistanceHidroloy',                                                   
                                                       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'])),
    ('dcf', DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.80)),    
]   
pipeline_1 = Pipeline(pipeline_list_1)
X_train_pipe_1 = pipeline_1.fit_transform(X_train)
X_train_pipe_1.head()

In [ ]:
X_test_pipe_1 = pipeline_1.transform(X_test)
X_test_pipe_1.head()


# Pipeline 2
In this pipeline I will change some transformers as :

* Outliers in Elevation will be windorized
* EuclidianDiscanteHidrology will not be Discretized, its outliers willbe windorized.
* Hillshade_9am will not be Discretized, its outliers will be windorized.
* Hillshade_Noon will not be Discretized, its outliers will be windorized.
* Hillshade_3pm will not be Discretized, its outliers will be windorized

In [ ]:
pipeline_list_2 = [
    ('dropuniquefeatures', DropIdentifierFeatures()),
    ('aspect', AspectTransformer(column='Aspect', new_name='Orientation')),
    ('soil_columns_dummies', FromDummiesToCategories(cols_to_operate=soil_columns, new_column_name='Soil_Type')),
    ('wilder_columns_dummies', FromDummiesToCategories(cols_to_operate=wilder_columns, new_column_name='Wilderness')),
    ('soil_rare', RareLabelEncoder(tol=0.05, variables=['Soil_Type'])),
    ('dp', DropConstantFeatures(tol=0.99)),
    ('dd', DropDuplicateFeatures()),
    ('dt', DistanceTransformer()),
     ('one_hot',OneHotEncoder(variables=['Soil_Type', 'Wilderness', 'Orientation'])),
    ('winds', Winsorizer(variables=['EuclideanDistanceHidroloy',  'Elevation',                                                 
                                                       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'])),    
    ('dcf', DropCorrelatedFeatures(variables=None, method='pearson', threshold=0.80)),    
]   
pipeline_2 = Pipeline(pipeline_list_2)
X_train_pipe_2 = pipeline_2.fit_transform(X_train)
X_train_pipe_2.head()

# Pipeline 3
This pipeline is similar to pipeline2 except it uses SmartcorrelatedSelection.

In [ ]:

pipeline_list_3 = [
    ('dropuniquefeatures', DropIdentifierFeatures()),
    ('aspect', AspectTransformer(column='Aspect', new_name='Orientation')),
    ('soil_columns_dummies', FromDummiesToCategories(cols_to_operate=soil_columns, new_column_name='Soil_Type')),
    ('wilder_columns_dummies', FromDummiesToCategories(cols_to_operate=wilder_columns, new_column_name='Wilderness')),
    ('soil_rare', RareLabelEncoder(tol=0.05, variables=['Soil_Type'])),
    ('dp', DropConstantFeatures(tol=0.99)),
    ('dt', DistanceTransformer()),
     ('one_hot',OneHotEncoder(variables=['Soil_Type', 'Wilderness', 'Orientation'])),
    ('winds', Winsorizer(variables=['EuclideanDistanceHidroloy',  'Elevation',                                                 
                                                       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'])),    
    ('dd', SmartCorrelatedSelection()),  
]   
pipeline_3 = Pipeline(pipeline_list_3)
X_train_pipe_3 = pipeline_3.fit_transform(X_train)
X_train_pipe_3.head()

# Pipeline 4
This pipeline will drop Soil and wilderness columns

In [ ]:
pipeline_list_4 = [
    ('dropuniquefeatures', DropIdentifierFeatures()),
    ('dropwild_soil',DropFeatures(soil_columns + wilder_columns)),
    ('aspect', AspectTransformer(column='Aspect', new_name='Orientation')),
    ('dp', DropConstantFeatures(tol=0.99)),
    ('dt', DistanceTransformer()),
    ('one_hot',OneHotEncoder(variables=['Orientation'])),
    ('winds', Winsorizer(variables=[
        'EuclideanDistanceHidroloy',  'Elevation', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'])),    
    ('dd', SmartCorrelatedSelection()),  
]   
pipeline_4 = Pipeline(pipeline_list_4)
X_train_pipe_4 = pipeline_4.fit_transform(X_train)
X_train_pipe_4.head()

# Pipeline 5
This pipeline will use again the coluns of wilderness and soiltype, but will not make the aspect transformation.

In [ ]:
pipeline_list_5 = [
    ('dropuniquefeatures', DropIdentifierFeatures()),
    ('soil_columns_dummies', FromDummiesToCategories(cols_to_operate=soil_columns, new_column_name='Soil_Type')),
    ('wilder_columns_dummies', FromDummiesToCategories(cols_to_operate=wilder_columns, new_column_name='Wilderness')),
    ('soil_rare', RareLabelEncoder(tol=0.05, variables=['Soil_Type'])),
    ('dp', DropConstantFeatures(tol=0.99)),
    ('dt', DistanceTransformer()),
     ('one_hot',OneHotEncoder(variables=['Soil_Type', 'Wilderness'])),
    ('winds', Winsorizer(variables=['EuclideanDistanceHidroloy',  'Elevation',                                                 
                                                       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'])),    
    ('dd', SmartCorrelatedSelection()),  
]   
pipeline_5 = Pipeline(pipeline_list_5)
X_train_pipe_5 = pipeline_5.fit_transform(X_train)
X_train_pipe_5.head()

# Evaluation Pipeline 1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
pipe_list_1_rf = [('rf', RandomForestClassifier(random_state=42))]
pipe_1_rf = Pipeline(pipeline_list_1 + pipe_list_1_rf)
rfpg1 ={
    'rf__n_estimators': [100, 150, 200],
    'rf__max_depth': [60,80, None],
    'rf__min_samples_split': [2, 3, 4],
}
grid1_pipe_1 = GridSearchCV(pipe_1_rf, param_grid=rfpg1, cv=sss, n_jobs=-1, verbose=3)
grid1_pipe_1.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid1_pipe_1.best_score_)) 
print("Test set score: {:.2f}".format(grid1_pipe_1.score(X_test, y_test))) 
print("Best parameters: {}".format(grid1_pipe_1.best_params_))

In [ ]:
from sklearn import metrics
y_predict_pipe_1 = grid1_pipe_1.predict(X_test)
pd.DataFrame(metrics.confusion_matrix(y_test, y_predict_pipe_1, labels=y_test.unique().tolist()), columns =y_test.unique().tolist(), index = y_test.unique().tolist() )

In [ ]:

print(metrics.classification_report(y_test, y_predict_pipe_1, digits=3))

# # # # Analysis of classifier
This classifier has problems with cover type 2 as it miss classify a large portion of the samples, specially it gets confusse class 2 with class 1

# Evaluation pipeline 2


In [ ]:
pipe_list_2_rf = [('rf', RandomForestClassifier(random_state=42))]
pipe_2_rf = Pipeline(pipeline_list_2 + pipe_list_2_rf)
rfpg2 ={
    'rf__n_estimators': [100, 150, 200],
    'rf__max_depth': [60,80, None],
    'rf__min_samples_split': [2, 3, 4],
}
grid1_pipe_2 = GridSearchCV(pipe_2_rf, param_grid=rfpg2, cv=sss, n_jobs=-1, verbose=3)
grid1_pipe_2.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid1_pipe_2.best_score_)) 
print("Test set score: {:.2f}".format(grid1_pipe_2.score(X_test, y_test))) 
print("Best parameters: {}".format(grid1_pipe_2.best_params_))

In [ ]:
y_predict_pipe_2 = grid1_pipe_2.predict(X_test)
pd.DataFrame(metrics.confusion_matrix(y_test, y_predict_pipe_2, labels=y_test.unique().tolist()), columns =y_test.unique().tolist(), index = y_test.unique().tolist() )

In [ ]:
print(metrics.classification_report(y_test, y_predict_pipe_2, digits=3))

# Conclusion
Im not happy with the results in this one, but im gonna stay with pipeline number 1, if in the future I have a better idea or if in my study I see somthing that is worth to try on Data Analysis I will do it in this dataset as i Really enjoy working on it.

In [ ]:
test = pd.read_csv('../input/forest-cover-type-prediction/test.csv')
test.head()

In [ ]:

test['Cover_Type'] = grid1_pipe_1.predict(test)

In [ ]:

to_kaggle = test[['Id', 'Cover_Type']].copy()

In [ ]:

to_kaggle.to_csv('grid1_pipe_1', index=False)


y_predict_pipe_1 = grid1_pipe_1.predict(X_test)